In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [3]:
train_tape = "Z:/rppg/ubfc_datatape_450x8x8_train.h5"
valid_tape = "Z:/rppg/ubfc_datatape_450x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [2]:
m1 = M_1()
m1.compile(optimizer='Nadam', loss='mae')
m1.build(input_shape=(None, 450, 8, 8, 3))
print(f'Flops per frame:{get_flops(m1, input_sig=[tf.TensorSpec([1, 450, 8, 8, 3])])/450:.0f}')
m1.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:259597
Model: "m_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 450, 64)           12352     
_________________________________________________________________
spectral_transform (Spectral multiple                  82560     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  41024     
_________________________________________________________________
sequential_2 (Sequential)    (None, 450, 64)           256       
_________________________________________________________________
spectral_transform_1 (Spectr multiple                  49792     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  10272     
_________________

In [ ]:
m1.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m1_ubfc.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m1.load_weights('../weights/m1_ubfc.h5')
m1.cross()

In [4]:
m1.load_weights('../weights/m1_ubfc.h5')
m1.cross()

In [7]:
eval_on_dataset(test_set_CCNU, m1, 450, (8, 8), step=1, batch=32, save='../results/ours_UBFC_CCNU.h5')
get_metrics('../results/ours_UBFC_CCNU.h5', use_filter=False)

100%|██████████| 179/179 [00:26<00:00,  6.80it/s]


{'Sliding window': {'MAE': 18.427, 'RMSE': 31.746, 'R': 0.00632},
 'Whole video': {'MAE': 14.883, 'RMSE': 27.853, 'R': 0.03905}}

In [6]:
eval_on_dataset(test_set_CCNU_rPPG, m1, 450, (8, 8), step=1, batch=32, save='../results/ours_UBFC_CCNU_rPPG.h5')
get_metrics('../results/ours_UBFC_CCNU_rPPG.h5', use_filter=False)

100%|██████████| 54/54 [00:05<00:00,  9.36it/s]


{'Sliding window': {'MAE': 18.449, 'RMSE': 30.815, 'R': 0.07882},
 'Whole video': {'MAE': 15.093, 'RMSE': 26.096, 'R': 0.14105}}

In [5]:
eval_on_dataset(test_set_PURE, m1, 450, (8, 8), step=1, batch=32, save='../results/ours_UBFC_PURE.h5')
get_metrics('../results/ours_UBFC_PURE.h5',use_filter=False)

100%|██████████| 59/59 [00:05<00:00, 11.64it/s]


{'Sliding window': {'MAE': 29.05, 'RMSE': 46.286, 'R': 0.24464},
 'Whole video': {'MAE': 25.611, 'RMSE': 43.391, 'R': 0.24848}}